In [ ]:
import pandas as pd

In [ ]:
# Используем две библиотеки для геокодирования:
# 1 - геокодеры из проекта geopy
from geopy.geocoders import Nominatim, Yandex
# 2 - стандартная библиотека веб-запросов (для работы с geocode.xyz)
import requests

# И еще парочку:
# - для ограничения скорости запросов
from geopy.extra.rate_limiter import RateLimiter
# - для визуализации процесса
from tqdm import tqdm

In [ ]:
import yaml

with open("../api_keys/geocoding_api.yml", "r") as f:
    api_keys = yaml.safe_load(f)

In [ ]:
geolocator_osm = Nominatim(user_agent="py")
geolocator_ya = Yandex(user_agent = "py", api_key = api_keys.get("yandex_geocoding_api").get("api_key"))
geocode_osm = RateLimiter(geolocator_osm.geocode, min_delay_seconds=2)
geocode_ya = RateLimiter(geolocator_ya.geocode, min_delay_seconds=2)

In [ ]:
import json
from datetime import date

In [ ]:
with open("../state/state.json", "r") as infile:
    geocoding_state = json.load(infile)
geocoding_state

In [ ]:
task = pd.read_csv('../data/geocoded_' + geocoding_state['name'] + '.csv', encoding='utf-8', sep=',')

In [ ]:
if(date.fromisoformat(geocoding_state['last_date']) < date.today()):
    print('ok')
else:
    print('завтра')

In [ ]:
with open("../state/state.json", "r") as infile:
    geocoding_state = json.load(infile)

if(date.fromisoformat(geocoding_state['last_date']) < date.today()):
    print('Продолжаем парсить ', geocoding_state['name'], 'с индекса ', geocoding_state['parsed'])
    startIndex = geocoding_state['parsed']
    endIndex = startIndex + 998
    curIndex = startIndex
    for index, row in task.iterrows():
        if (index >= startIndex) and (index < endIndex):
            curIndex += 1
            name_for_osm = str.replace(row['formalname_city'] + ', ' + str(row['formalname_street']) + ', ' + str(row['house_number']) + str(row['building']) + str(row['block']) + str(row['letter']), 'nan', '')
            location_osm = geocode_osm(name_for_osm)
            print(name_for_osm)
            print('osm: ' + str(location_osm))
            location_ya = geocode_ya(row['address'])
            print(row['address'])
            print('ya: ' + str(location_ya))
            if location_osm:
                task.loc[index, 'addr_osm'] = location_osm.address
                task.loc[index, 'lon_osm'] = location_osm.longitude
                task.loc[index, 'lat_osm'] = location_osm.latitude
            if location_ya:
                task.loc[index, 'addr_ya'] = location_ya.address
                task.loc[index, 'lon_ya'] = location_ya.longitude
                task.loc[index, 'lat_ya'] = location_ya.latitude

    task.to_csv('../data/geocoded_' + geocoding_state['name'] + '.csv')

    geocoding_state['parsed'] = curIndex
    geocoding_state['last_date'] = date.today().isoformat()
    with open("../state/state.json", "w") as outfile:
        json.dump(geocoding_state, outfile, indent=2)
else :
    print('Подожди до завтра')